In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('links.csv')

In [18]:
df

,uuid,url,title,image,subtitle,content
0,0026af85-18b8-42a1-966a-17ac9d7ce9ea,https://www.rt.com/sport/552398-russian-gymnas...,Russian ‘Z’ symbol gymnast speaks out over fea...,https://mf.b37mrtl.ru/files/2022.03/xxs/623884...,"Ivan Kuliak said he did ‘nothing wrong’, while...",Russian gymnast Ivan Kuliak insists he should ...
1,026d11d8-98cb-4700-8fed-277eec712ba1,https://www.rt.com/sport/552445-evgeny-rylov-s...,Olympic champion stripped of Speedo deal after...,https://mf.b37mrtl.ru/files/2022.03/xxs/623987...,Evgeny Rylov appeared at the event in Moscow l...,Swimwear company Speedo has terminated its spo...
2,0217dc27-6c93-49f0-87ff-b8e83d959c7a,https://www.rt.com/sport/552931-russia-banned-...,Norway’s boycott of Russia caused safety fears...,https://mf.b37mrtl.ru/files/2022.03/xxs/624342...,The Norwegian Ski Federation issued a blunt st...,Russian skiers feared for their safety at even...
3,00bce962-4ca4-40e1-a869-7b25724acfbf,https://sputniknews.com/20220319/are-bidens-af...,Are Biden's Afghanistan & Ukraine Blunders Beh...,https://cdnn1.img.sputniknews.com/img/07e5/04/...,"On 16 March, The New York Times finally admitt...","""The NYT piece on Hunter Biden's laptop comes ..."
4,018d31cd-bf19-4afe-b71c-9d2d1cb163e6,https://www.rt.com/news/552914-cyberattacks-uk...,Russia blames US for cyberattacks,https://mf.b37mrtl.ru/files/2022.03/xxs/624310...,Moscow repels hundreds of thousands of cyberat...,The US and its allies have put together a grou...
...,...,...,...,...,...,...
961,fa86b42f-646b-42ce-896e-4939e94aa890,https://www.rt.com/russia/552285-donbass-russi...,,,,
962,ffc94fc0-8cec-40cd-a80f-c3228feb93de,https://www.rt.com/shows/documentary/553005-az...,,,,
963,ffdc483a-91b0-4d5e-88db-9fcaa8aa9518,https://www.rt.com/news/552652-pope-women-sanc...,,,,
964,39a479c7-6efa-4bf4-a1a2-5880fb8e369d,https://www.rt.com/russia/552600-diplomats-pre...,,,,


In [4]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [41]:
def getArticleText(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    image = ''
    media = soup.find('div',class_='photoview__open')
    if(media):
        image = media['data-photoview-src']
    title = ''
    h1 = soup.select('h1.article__title')
    if(len(h1)>0):
        title = h1[0].text.strip()
    subtitle = ''
    st = soup.find('div', class_='article__announce-text')
    if(st is not None):
        subtitle = st.decode_contents()
    content = ''
    texts = soup.findAll('div',class_='article__text')
    for text in texts:
        content += cleanhtml(text.decode_contents()) + ' '
    print('title: ', title)
#     print('image: ', image)
#     print('subtitle: ', subtitle)
#     print('content: ', content)
    return title, image, subtitle, content

In [39]:
def getRtArticleText(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    image = ''
    media = soup.find('picture')
    if(media is not None and media.img['data-src']):
        image = media.img['data-src']
    title = ''
    h1 = soup.select('h1.article__heading')
    if(len(h1)>0):
        title = h1[0].decode_contents().strip()
    subtitle = ''
    summary = soup.find('div', class_='article__summary summary')
    if(summary is not None):
        subtitle = summary.decode_contents().strip()
    divs = soup.find('div',class_='article__text text')
    texts = []
    if(divs is not None):
        texts = divs.findAll('p')
    content = ''
    if(len(texts)>0):
        for t in texts:
             content += t.text
    
    print('title: ', title)
#     print('image: ', image)
#     print('subtitle: ', subtitle)
#     print('content: ', content)
    return title, image, subtitle, content
# getRtArticleText(link)

In [42]:
for index, row in df.iterrows():
    if(row.title == ''):
        print(index, ' ', row.url)
        link = row.url
        cnt = link.count('https')
        if(row.url.__contains__('https://sputniknews.com')):
            if(cnt>1):
                idx = link.rfind('https://sputniknews.com')
                link = link[idx:]
            t, i, s, c = getArticleText(link)
            df.at[index,'title'] = t
            df.at[index,'image'] = i
            df.at[index,'subtitle'] = s
            df.at[index,'content'] = c
        elif(row.url.__contains__('https://www.rt.com')):
            if(cnt>1):
                idx = link.rfind('https://www.rt.com')
                link = link[idx:]
            t, i, s, c = getRtArticleText(link)
            df.at[index,'title'] = t
            df.at[index,'image'] = i
            df.at[index,'subtitle'] = s
            df.at[index,'content'] = c

21   https://is.gd/yrkVVG
23   http://dlvr.it/SMF8lJ
29   http://dlvr.it/SMhLLT
41   http://dlvr.it/SMgVCq
51   http://dlvr.it/SMPN2J
52   http://dlvr.it/SMXGMk
55   http://dlvr.it/SMShWx
61   https://mid.ru/en/foreign_policy/news/1807028/
63   http://dlvr.it/SMZJ44
76   http://dlvr.it/SMNXws
84   https://www.urdupoint.com/en/world/russias-upper-house-to-take-part-in-joint-pr-1486173.html
87   http://dlvr.it/SM8bck
92   https://bit.ly/3qhyyRk
106   http://dlvr.it/SMVWCB
135   http://dlvr.it/SM7Dj4
143   http://dlvr.it/SMhBLK
149   http://dlvr.it/SMgddq
174   http://dlvr.it/SMGSRK
200   https://is.gd/tkGUeH
204   https://is.gd/zXq9qy
210   http://dlvr.it/SM57kV
217   http://dlvr.it/SMRD7n
225   http://dlvr.it/SMQ44h
227   https://support.twitter.com/articles/20169199
230   http://dlvr.it/SMFdTd
231   https://en.wikipedia.org/wiki/Lviv_pogroms_(1941)
259   http://dlvr.it/SMC8sr
266   http://dlvr.it/SMRHxx
271   http://dlvr.it/SMc89J
277   http://dlvr.it/SM453j
278   http://dlvr.it/SMj3N5

title:  'World's Strongest Girl' Responds to Schwarzenegger's Message to Russian People About Ukraine
526   https://sputniknews.com/20220330/le-pen-gains-3-points-in-a-week-closes-in-on--macron-ahead-of-french-election-show-polls-1094320373.html
title:  Le Pen Gains 3 Points in a Week, Closes in on  Macron Ahead of French Election, Show Polls
527   https://www.rt.com/russia/553049-kremlin-putin-military-claims/
title:  US doesn't understand how Russia works – Kremlin
528   https://sputniknews.com/20220323/us-treasury-secretary-to-discuss-freezing-russian-gold-reserves-with-senators-report-says-1094113900.html
title:  US Treasury Secretary to Discuss Freezing Russian Gold Reserves With Senators, Report Says
529   https://sputniknews.com/20220323/lavrov-the-current-russophobia-in-the-west-is-a-verdict-on-western-civilisation-1094108709.html
title:  Lavrov: The Current Russophobia in the West is a Verdict on Western Civilisation
530   https://sputniknews.com/20220325/erdogan-on-idea-of-gi

title:  Did Biden Spill Beans About Secret Plans to Train Ukrainian Troops & Put US Boots in Ukraine?
574   http://dlvr.it/SMZR6G
575   https://sputniknews.com/20220328/ex-sbu-member-says-kiev-tried-to-hide-evidence-of-black-site-prison-at-mariupol-airport--video-1094255138.html
title:  Ex-SBU Member Says Kiev Tried to Hide Evidence of Black Site Prison at Mariupol Airport – Video
576   https://www.rt.com/news/552636-stoltenberg-stays-nato-ukraine/
title:  NATO chief extends term over Ukraine war
577   https://www.rt.com/news/552407-german-truckers-fuel-price-protest/
title:  German truckers protest rising fuel prices (VIDEO)
578   https://sputniknews.com/20220319/watch-russian-ka-52-helicopter-destroy-fortified-bases-of-ukrainian-nationalists-1094003957.html
title:  WATCH: Russian Ka-52 Helicopter Destroys Fortified Bases of Ukrainian Nationalists
579   https://sputniknews.com/20220331/putin-western-countries-must-open-accounts-in-russian-banks-to-buy-russian-gas-1094359921.html
title

title:  CrossTalk Bullhorns, HOME EDITION: Biden’s war
629   https://sputniknews.com/20220318/bidens-insistence-putin-a-war-criminal-met-with-skepticism-1093994985.html
title:  Biden's Insistence Putin a 'War Criminal' Met With Skepticism
630   http://bit.ly/3L689On
631   https://www.rt.com/news/552283-czech-ex-president-refuses-return-russian-award/
title:  Former Czech president refuses to return Russian award
632   https://sputniknews.com/20220330/azov-battalion-commander-tells-us-newspaper-majority-of-ukrainians-today-are-nationalists-1094309072.html
title:  Azov Battalion Commander Tells US Newspaper ‘Majority of Ukrainians Today Are Nationalists’
633   https://www.rt.com/russia/553032-ukraine-arms-wishlist-revealed/
title:  Ukraine sends new arms ‘wish list’ to US – media
634   https://sputniknews.com/20220321/oil-embargo-would-hit-energy-balance-in-europe-kremlin-warns-1094045658.html
title:  Oil Embargo Would Hit Energy Balance in Europe, Kremlin Warns
635   https://www.rt.com/

title:  Figure skating showpiece ‘boring’ without Russians – Olympic queen
683   https://www.rt.com/sport/552903-irina-rodnina-gabriella-papadakis-comments/
title:  Competitions ‘easier’ with Russians banned – skating icon
684   https://sputniknews.com/20220322/belarus-kgb-says-eight-out-of-19-diplomats-at-ukraine-embassy-are-spies---report-1094093317.html
title:  Belarus KGB Says Eight Out of 19 Diplomats at Ukraine Embassy Are Spies - Report
685   https://www.rt.com/shows/worlds-apart-oksana-boyko/552767-pavan-varma-military-confrontation/
title:  Silence of violence? Pavan Varma, Indian career diplomat
686   http://dlvr.it/SM8Wlg
687   https://www.rt.com/news/552344-british-soldier-arrested-ukraine/?utm_source=Newsletter&utm_medium=Email&utm_campaign=Email
title:  Elite British soldier arrested after going AWOL to fight in Ukraine – media
688   https://www.rt.com/sport/552877-russia-suspended-biathlon-ibu/
title:  Sports federation accused of ‘leading anti-Russian front’
689   https

title:  Russian Envoy Warns Ukraine Against Using Civilians as Human Shields After UNSC Drops Resolution
737   https://on.rt.com/bu8s
738   https://www.rt.com/news/552493-capitol-riot-granted-asylum-belarus/
title:  Capitol riot suspect receives asylum in Belarus
739   https://www.rt.com/russia/552533-us-attempts-freeze-gold-reserves/
title:  US Senators want to freeze Russia’s gold reserves – media
740   https://sputniknews.com/20220326/ambassador-antonov-russia-unlike-the-us-does-everything-to-stop-war-crimes-1094209870.html
title:  Ambassador Antonov: Russia, Unlike the US, Does Everything to Stop War Crimes
741   https://sputniknews.com/20220321/us-could-expand-sanctions-against-russia-nowhere-near-lifting-restrictions-1094039487.html
title:  US Threatens to Expand Sanctions Against Russia, Says ‘Nowhere Near’ Lifting Restrictions
742   http://dlvr.it/SMd8Dl
743   https://sputniknews.com/20220331/us-to-release-1-million-barrels-of-oil-a-day-from-strategic-reserves-over-next-six-mon

title:  Poland wants to confiscate Russian-owned property
790   https://www.rt.com/business/552859-eu-confirms-dependence-russia-energy/
title:  EU estimates date of its energy independence from Russia
791   http://dlvr.it/SMj8nM
792   https://www.rt.com/news/552777-iran-jcpoa-potentially-revived-borrell/
title:  EU foreign policy chief says Iran nuke deal ‘very close’
793   https://sputniknews.com/20220330/bidens-budget-poisoned-negotiations-prison-services-corruption-1094307654.html
title:  Biden’s Budget, Poisoned Negotiations, Prison Services Corruption
794   http://dlvr.it/SMg8XB
795   https://sputniknews.com/20220324/fund-of-joe-bidens-son-connected-with-financing-of-us-military-bio-activities-in-ukraine---mod-1094150901.html
title:  Hunter Biden’s Investment Fund Connected to Financing of Pentagon-Funded Biolabs in Ukraine: MoD
796   https://sputniknews.com/20220322/turkish-defence-ministry-reportedly-warned-ships-of-mines-drifting-in-black-sea-1094078786.html
title:  Turkish De

title:  South Korea seeks to continue trade with Russia despite sanctions
845   https://www.rt.com/russia/552628-nato-ukraine-support-stoltenberg/
title:  NATO explains why it won't send troops to Ukraine
846   https://sputniknews.com/20220326/hungarian-pm-orban-lashes-back-at-zelensky-over-criticism-for-not-backing-anti-russia-sanctions-1094218498.html
title:  Hungarian PM Orban Lashes Back at Zelensky Over Criticism for Not Backing Anti-Russia Sanctions
847   http://dlvr.it/SM3hSy
848   http://dlvr.it/SMCmzc
849   https://sputniknews.com/20220320/enemy-of-humanity-kim-dotcom-slams-us-plans-to-deploy-missiles-in-taiwan-okinawa-philippines-1094033515.html
title:  'Enemy of Humanity': Kim Dotcom Slams US Plans to Deploy Missiles in Taiwan, Okinawa, Philippines
850   https://www.rt.com/sport/552995-perm-basketball-arena-russia-sanctions-europe/
title:  Russian region shuns European equipment for sports arena, turns to China
851   http://dlvr.it/SMYdJn
852   https://sputniknews.com/202203

title:  Post-Abramovich era edges closer as Chelsea sale shortlist takes shape
905   https://sputniknews.com/20220322/biden-cant-do-much-to-help-europe-with-unrealistic-push-away-from-russian-energy-supplies-report-1094091598.html
title:  Biden Can’t Do Much to Help Europe With ‘Unrealistic’ Push Away From Russian Energy Supplies: Report
906   https://sputniknews.com/20220327/lpr-head-says-republic-may-hold-referendum-on-joining-russia-soon-1094232866.html
title:  Lugansk People's Republic May Hold Referendum on Joining Russia Soon
907   https://www.rt.com/news/552626-ukraine-conflict-living-standard/
title:  Western elites are exploiting the Ukraine conflict to lower the standard of living
908   https://sputniknews.com/20220327/uk-claims-it-gets-just-3-per-cent-of-its-gas-from-russia--and-none-via-pipelines-1094244313.html
title:  UK Claims It Gets Just 3 Per Cent of Its Gas From Russia — And None via Pipelines
909   https://sputniknews.com/20220322/russia-would-only-use-nuclear-weapo

title:  Banned Olympic ice champs to perform in Russia during World Championships
955   https://mid.ru/en/foreign_policy/news/1806797/
956   https://sputniknews.com/20220320/france-freezes-24bln-of-russian-central-banks-assets-1094034486.html
title:  France Freezes $24Bln of Russian Central Bank's Assets
957   https://www.rt.com/sport/552802-chelsea-april-takeover-deadline/
title:  Chelsea takeover set to drag on until April as bidders given extended deadline
958   http://dlvr.it/SMbvZ4
959   https://www.rt.com/russia/552939-chechen-mariupol-nemtsov-killing/
title:  Rumored suspect in Russian opposition figure murder wounded in Ukraine
960   https://sputniknews.com/20220329/russian-special-forces-capture-ukrainian-nationalists-who-tortured-russian-soldiers-lawmaker-says-1094300874.html
title:  Russian Special Forces Capture Ukrainian Nationalists Who Tortured Russian Soldiers, Lawmaker Says
961   https://www.rt.com/russia/552285-donbass-russia-ukraine-history/
title:  Historic roots of

In [23]:
df.iloc[17]['url']

'https://archive.ph/2022.03.30-103137/https://sputniknews.com/20220330/synagogue-in-ukraines-uman-was-used-to-house-nazis-their-weapons-says-russian-defense-ministry-1094315701.html'

In [24]:
link = 'https://archive.ph/2022.03.30-103137/https://sputniknews.com/20220330/synagogue-in-ukraines-uman-was-used-to-house-nazis-their-weapons-says-russian-defense-ministry-1094315701.html'

In [27]:
idx = link.rfind('https://sputniknews.com')
link[idx:]
print(idx)

37


In [28]:
print(link)

https://archive.ph/2022.03.30-103137/https://sputniknews.com/20220330/synagogue-in-ukraines-uman-was-used-to-house-nazis-their-weapons-says-russian-defense-ministry-1094315701.html


In [43]:
df

,uuid,url,title,image,subtitle,content
0,0026af85-18b8-42a1-966a-17ac9d7ce9ea,https://www.rt.com/sport/552398-russian-gymnas...,Russian ‘Z’ symbol gymnast speaks out over fea...,https://mf.b37mrtl.ru/files/2022.03/xxs/623884...,"Ivan Kuliak said he did ‘nothing wrong’, while...",Russian gymnast Ivan Kuliak insists he should ...
1,026d11d8-98cb-4700-8fed-277eec712ba1,https://www.rt.com/sport/552445-evgeny-rylov-s...,Olympic champion stripped of Speedo deal after...,https://mf.b37mrtl.ru/files/2022.03/xxs/623987...,Evgeny Rylov appeared at the event in Moscow l...,Swimwear company Speedo has terminated its spo...
2,0217dc27-6c93-49f0-87ff-b8e83d959c7a,https://www.rt.com/sport/552931-russia-banned-...,Norway’s boycott of Russia caused safety fears...,https://mf.b37mrtl.ru/files/2022.03/xxs/624342...,The Norwegian Ski Federation issued a blunt st...,Russian skiers feared for their safety at even...
3,00bce962-4ca4-40e1-a869-7b25724acfbf,https://sputniknews.com/20220319/are-bidens-af...,Are Biden's Afghanistan & Ukraine Blunders Beh...,https://cdnn1.img.sputniknews.com/img/07e5/04/...,"On 16 March, The New York Times finally admitt...","""The NYT piece on Hunter Biden's laptop comes ..."
4,018d31cd-bf19-4afe-b71c-9d2d1cb163e6,https://www.rt.com/news/552914-cyberattacks-uk...,Russia blames US for cyberattacks,https://mf.b37mrtl.ru/files/2022.03/xxs/624310...,Moscow repels hundreds of thousands of cyberat...,The US and its allies have put together a grou...
...,...,...,...,...,...,...
961,fa86b42f-646b-42ce-896e-4939e94aa890,https://www.rt.com/russia/552285-donbass-russi...,Historic roots of the Donbass problem explained,https://mf.b37mrtl.ru/files/2022.03/xxs/6235b3...,The region has a distinct identity and doesn’t...,"Originally published on March 19, 2022.Current..."
962,ffc94fc0-8cec-40cd-a80f-c3228feb93de,https://www.rt.com/shows/documentary/553005-az...,Ukraine: Heading East,,,Donbass residents have fled southeast Ukraine ...
963,ffdc483a-91b0-4d5e-88db-9fcaa8aa9518,https://www.rt.com/news/552652-pope-women-sanc...,"Pope speaks against sanctions, calls for more ...",https://mf.b37mrtl.ru/files/2022.03/xxs/623cb9...,The Pontiff said women could convert power fro...,"Women must change the way society is governed,..."
964,39a479c7-6efa-4bf4-a1a2-5880fb8e369d,https://www.rt.com/russia/552600-diplomats-pre...,Experts warned for decades that NATO expansion...,https://mf.b37mrtl.ru/files/2022.03/xxs/623c35...,Analysts and diplomats have been saying since ...,Russia’s military offensive in Ukraine has bee...


In [52]:
df.to_csv('scraping_big.csv',index=False)

In [45]:
df['title'].isna().sum()

0

In [50]:
df = df[df['title'] != ""]

In [51]:
df

,uuid,url,title,image,subtitle,content
0,0026af85-18b8-42a1-966a-17ac9d7ce9ea,https://www.rt.com/sport/552398-russian-gymnas...,Russian ‘Z’ symbol gymnast speaks out over fea...,https://mf.b37mrtl.ru/files/2022.03/xxs/623884...,"Ivan Kuliak said he did ‘nothing wrong’, while...",Russian gymnast Ivan Kuliak insists he should ...
1,026d11d8-98cb-4700-8fed-277eec712ba1,https://www.rt.com/sport/552445-evgeny-rylov-s...,Olympic champion stripped of Speedo deal after...,https://mf.b37mrtl.ru/files/2022.03/xxs/623987...,Evgeny Rylov appeared at the event in Moscow l...,Swimwear company Speedo has terminated its spo...
2,0217dc27-6c93-49f0-87ff-b8e83d959c7a,https://www.rt.com/sport/552931-russia-banned-...,Norway’s boycott of Russia caused safety fears...,https://mf.b37mrtl.ru/files/2022.03/xxs/624342...,The Norwegian Ski Federation issued a blunt st...,Russian skiers feared for their safety at even...
3,00bce962-4ca4-40e1-a869-7b25724acfbf,https://sputniknews.com/20220319/are-bidens-af...,Are Biden's Afghanistan & Ukraine Blunders Beh...,https://cdnn1.img.sputniknews.com/img/07e5/04/...,"On 16 March, The New York Times finally admitt...","""The NYT piece on Hunter Biden's laptop comes ..."
4,018d31cd-bf19-4afe-b71c-9d2d1cb163e6,https://www.rt.com/news/552914-cyberattacks-uk...,Russia blames US for cyberattacks,https://mf.b37mrtl.ru/files/2022.03/xxs/624310...,Moscow repels hundreds of thousands of cyberat...,The US and its allies have put together a grou...
...,...,...,...,...,...,...
961,fa86b42f-646b-42ce-896e-4939e94aa890,https://www.rt.com/russia/552285-donbass-russi...,Historic roots of the Donbass problem explained,https://mf.b37mrtl.ru/files/2022.03/xxs/6235b3...,The region has a distinct identity and doesn’t...,"Originally published on March 19, 2022.Current..."
962,ffc94fc0-8cec-40cd-a80f-c3228feb93de,https://www.rt.com/shows/documentary/553005-az...,Ukraine: Heading East,,,Donbass residents have fled southeast Ukraine ...
963,ffdc483a-91b0-4d5e-88db-9fcaa8aa9518,https://www.rt.com/news/552652-pope-women-sanc...,"Pope speaks against sanctions, calls for more ...",https://mf.b37mrtl.ru/files/2022.03/xxs/623cb9...,The Pontiff said women could convert power fro...,"Women must change the way society is governed,..."
964,39a479c7-6efa-4bf4-a1a2-5880fb8e369d,https://www.rt.com/russia/552600-diplomats-pre...,Experts warned for decades that NATO expansion...,https://mf.b37mrtl.ru/files/2022.03/xxs/623c35...,Analysts and diplomats have been saying since ...,Russia’s military offensive in Ukraine has bee...
